In [1]:
import numpy as np

In [2]:
class Vertice:
  def __init__(self, numero, adj_list_entrada, adj_list_saida):
    self.numero = numero
    self.adj_list_entrada = adj_list_entrada
    self.adj_list_saida = adj_list_saida

  def numero(self):
    return self.numero


In [3]:
class Grafo:
  def __init__(self, matriz_arestas_passada):
    self.matriz_arestas = np.copy(matriz_arestas_passada)
    qtd_vertices = np.shape(matriz_arestas)[0]
    if(qtd_vertices > 0 and np.shape(matriz_arestas)[0] == np.shape(matriz_arestas)[1]):
      self.vertices = []
      for i in range(qtd_vertices):
        A = Vertice(i+1, [], [])
        self.vertices.append(A)
      for i in range(qtd_vertices):
        for j in range(qtd_vertices):
          if(matriz_arestas[i][j] > 0 and matriz_arestas[i][j] != np.inf):
            for k in self.vertices:
              if(k.numero == j+1):
                aux2 = k
              if(k.numero == i+1):
                aux1 = k
            aux1.adj_list_saida.append(aux2.numero)
            aux2.adj_list_entrada.append(aux1.numero)

  def quantidadeVertices(self):
    return len(self.vertices)

  def quantidadeArestas(self):
    return len(self.arestas)



In [4]:
def gerarMatriz(linhas, colunas):
  result = []
  colunas = colunas + 1 #Para gerar uma coluna a mais, pois vou pular a posição 0
  for i in range(linhas):
    a = []
    for j in range(colunas):
      a.append(np.inf)
    result.append(a)
  return result


In [5]:
def BellmanFord(G, v):
  n = G.quantidadeVertices()
  C = gerarMatriz(n,n)
  C = np.array(C)
  C[0][v] = 0;
  #print(C)
  for i in range(1, n):
    if(i != v):
      C[0][i] = np.inf

  for l in range(1, n):
    for i in range(1, n+1):
      lista = []
      min_entrada = np.inf
      #print("i:{}".format(i))
      for j in G.vertices[i-1].adj_list_entrada:
        #print("C(l-1, j)+w(j, i): {}".format(C[l-1][j] + G.matriz_arestas[j-1][i-1]))
        lista.append(C[l-1][j] + G.matriz_arestas[j-1][i-1])
      if(len(lista) > 0):
        min_entrada = min(lista)
      #print("Min:{}".format(min_entrada))
      if(min_entrada < C[l-1][i]):
        C[l][i] = min_entrada
      else:
        C[l][i] = C[l-1][i]

  return C[n-1][1:]

In [6]:
def FloydWarshall(G):
  n = G.quantidadeVertices()
  conj_matrizes = []
  A = G.matriz_arestas.copy()
  conj_matrizes.append(A)

  for k in range(1, n):
    W = conj_matrizes[len(conj_matrizes)-1].copy()
    for i in range(n):
      for j in range(n):
        lista = []
        lista.append(conj_matrizes[len(conj_matrizes)-1][i][j])
        lista.append(conj_matrizes[len(conj_matrizes)-1][i][k]+conj_matrizes[len(conj_matrizes)-1][k][j])
        W[i][j] = min(lista)
        conj_matrizes.append(W)

  return conj_matrizes[len(conj_matrizes)-1]

In [7]:
matriz_arestas = [[0, 1, np.inf ,np.inf],
                  [np.inf, 0, 1, np.inf],
                  [np.inf, np.inf, 0, 1],
                  [1, 1, np.inf, 0]]
G = Grafo(matriz_arestas)

In [8]:
from heapq import heapify, heappush, heappop

In [9]:
def Djkstra(G, v):
  n = G.quantidadeVertices()
  C = []
  C.append(-1) #APENAS PARA PULAR O INDICE 0
  #ADICIONANDO OS CUSTOS INICIAIS
  for i in range(n):
    if(i != v-1):
      C.append(G.matriz_arestas[v-1][i])
    else:
      C.append(0)

  #VETOR INDICATIVO SE UM VÉRTICE AINDA ESTÁ NA HEAP
  pertence_heap = [True] * n

  #CRIANDO A HEAP
  heap = []
  heapify(heap)

  #ADICIONANDO VALORES NA HEAP
  for i in range(n+1):
    if(i != 0):
      heappush(heap, C[i])

  while( len(heap) != 0 ):
    j = heappop(heap)
    for i in range(1, n+1):
      if C[i] == j and pertence_heap[i-1] == True:
        pertence_heap[i-1] = False
        num_vet = i

    for i in G.vertices[num_vet-1].adj_list_saida:
      if(pertence_heap[i-1] == True):
        if(C[i] > C[num_vet] + G.matriz_arestas[num_vet-1][i-1]):
          for k in range(len(heap)):
            if heap[k] == C[i]:
              heap[k] = C[num_vet] + G.matriz_arestas[num_vet-1][i-1]
              break
          C[i] = C[num_vet] + G.matriz_arestas[num_vet-1][i-1]

  return C[1:]


In [10]:
A = Djkstra(G, 4)
A

[1.0, 1.0, 2.0, 0]

In [11]:
B = BellmanFord(G, 1)
B

array([0., 1., 2., 3.])

In [12]:
C = FloydWarshall(G)
C

array([[0., 1., 2., 3.],
       [3., 0., 1., 2.],
       [2., 2., 0., 1.],
       [1., 1., 2., 0.]])